In [1]:
from qiskit import *
import numpy as np

In [2]:
# define the search list and target value

search_list = [3, 0, 1, 2]
target_value = 2

In [3]:
# convert target value to binary (2-bit representation)

target_binary = format(target_value, '02b')
print(f"Target value {target_value} in binary: |{target_binary}⟩")

Target value 2 in binary: |10⟩


In [4]:
# create oracle circuit for marking the target value's binary representation

oracle = QuantumCircuit(2, name='oracle')

if target_binary == '11':
    oracle.cz(0, 1)
elif target_binary == '01':
    oracle.x(1)
    oracle.cz(0, 1)
    oracle.x(1)
elif target_binary == '10':
    oracle.x(0)
    oracle.cz(0, 1)
    oracle.x(0)
elif target_binary == '00':
    oracle.x([0, 1])
    oracle.cz(0, 1)
    oracle.x([0, 1])
oracle_gate = oracle.to_gate()

oracle.draw()

┌───┐   ┌───┐
q_0: ┤ X ├─■─┤ X ├
     └───┘ │ └───┘
q_1: ──────■──────

In [5]:
# create reflection operator (diffusion operator)

reflection = QuantumCircuit(2, name='reflection')
reflection.h([0, 1])
reflection.z([0, 1])
reflection.cz(0, 1)
reflection.h([0, 1])
reflection_gate = reflection.to_gate()

reflection.draw()

┌───┐┌───┐   ┌───┐
q_0: ┤ H ├┤ Z ├─■─┤ H ├
     ├───┤├───┤ │ ├───┤
q_1: ┤ H ├┤ Z ├─■─┤ H ├
     └───┘└───┘   └───┘

In [6]:
# create main grover circuit
grover_circ = QuantumCircuit(2, 2)


grover_circ.h([0, 1])
grover_circ.append(oracle_gate, [0, 1])
grover_circ.append(reflection_gate, [0, 1])

# measure qubits
grover_circ.measure([0, 1], [0, 1])

grover_circ.draw()

┌───┐┌─────────┐┌─────────────┐┌─┐   
q_0: ┤ H ├┤0        ├┤0            ├┤M├───
     ├───┤│  oracle ││  reflection │└╥┘┌─┐
q_1: ┤ H ├┤1        ├┤1            ├─╫─┤M├
     └───┘└─────────┘└─────────────┘ ║ └╥┘
c: 2/════════════════════════════════╩══╩═
                                     0  1

In [7]:
# execute

backend = Aer.get_backend('qasm_simulator')
job = execute(grover_circ, backend, shots=1000)
result = job.result()
counts = result.get_counts()

In [8]:
# display results

print("\nValue to Binary State Mapping:")
print("-" * 50)
for value in search_list:
    binary = format(value, '02b')
    print(f"Value {value}: |{binary}⟩")

print("\nMeasurement Results:")
print("-" * 50)
for state in ['00', '01', '10', '11']:
    count = counts.get(state, 0)
    value = int(state, 2)
    print(f"State |{state}⟩: {count:5d} counts (Value: {value})")

# find the most frequent state
max_state = max(counts.items(), key=lambda x: x[1])

# convert binary state directly to value
found_value = int(max_state[0], 2)

print("\nSearch Results:")
print("-" * 50)
print(f"Most frequent state: |{max_state[0]}⟩")
print(f"Found value: {found_value}")
print(f"Target value was: {target_value}")
print(f"Search {'successful' if found_value == target_value else 'failed'}")


Value to Binary State Mapping:
--------------------------------------------------
Value 3: |11⟩
Value 0: |00⟩
Value 1: |01⟩
Value 2: |10⟩

Measurement Results:
--------------------------------------------------
State |00⟩:     0 counts (Value: 0)
State |01⟩:     0 counts (Value: 1)
State |10⟩:  1000 counts (Value: 2)
State |11⟩:     0 counts (Value: 3)

Search Results:
--------------------------------------------------
Most frequent state: |10⟩
Found value: 2
Target value was: 2
Search successful
